## Resume

In [ ]:
import csv
from time import sleep
from getpass import getpass

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [ ]:
driver = Chrome()
driver.get('https://www.twitter.com/login')
driver.maximize_window()

In [ ]:
username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('cgomezloaiza@gmail.com')
my_password = getpass()


password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)
sleep(1)

In [ ]:
profile = 'BillGates'

In [ ]:
driver.get('https://www.twitter.com/'+ profile)

In [ ]:
cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')

In [ ]:
card = cards[0]

In [ ]:
card.find_element_by_xpath('./div[2]/div[1]//span').text

In [ ]:
def get_tweet_data(card):
    username = card.find_element_by_xpath('.//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(),"@")]').text
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element_by_xpath('./div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('./div[2]/div[2]/div[2]').text
    text = comment + responding
    
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    
    return tweet

In [ ]:
get_tweet_data(card)

In [ ]:
tweet_data = []

for card in cards:
    data = get_tweet_data(card)
    if data:
        tweet_data.append(data)

In [ ]:
card.find_element_by_xpath('.//div[@data-testid="retweet"]').text

In [ ]:
tweet_data

In [ ]:
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == 4:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 4:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = 4
            break

In [ ]:
data[0]

In [ ]:
with open('tweets_table.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("tweets_table.csv")

In [ ]:
df.head(100)